<a href="https://colab.research.google.com/github/uomodellamansarda/clockworkcsvestimator/blob/master/Workout_Analyzer_Eng_V0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOTE
If the time series is not stationary, or any of its possible trasformations you could not evaluate autocorrelation and partial correlation 

Modifiche al codice, nel caso di nuovo anno bisogna capire come andare a ritroso nel tempo di tre-sei mesi altrimenti viene generato un errore 
Basterebbe selezionare differenti funzioni. 
Indubbiamente adesso bisogna rieditare il codice perchè si è capito che per la selezione basta fare 
df.iloc[start:]


In the Year, Month, Week Analysis, I should create a logic that starts counting from the current day, instead at the moment it starts counting from the last log

#1 Library Setup


In [ ]:
#Google Drive Pre-work
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#importing all the necessary libraries 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import scipy.stats

#DataLoading

In [ ]:
#global variable of interest 

va='Python'
tvoi='Length'

#goal 
h=10000
#h1mbW7YyNbQV3M7LgTVxWeR_rt4tjA2PfZ
namepath="https://drive.google.com/file/d/1mbW7YyNbQV3M7LgTVxWeR_rt4tjA2PfZ/view?usp=sharing"
id="1mbW7YyNbQV3M7LgTVxWeR_rt4tjA2PfZ"
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  

# Dataset is now stored in a Pandas Dataframe
csvname="Filename.csv"
#read the csv	
columns_name=['Year', 'Month', 'Day', 'Time', 'Length', 'Start', 'End', 'Activity']
dfraw=pd.read_csv(csvname,names=columns_name,sep=',',skiprows=1,skipfooter=0, index_col=False)

print(type(dfraw[tvoi].iloc[0]))
dfraw[tvoi] = dfraw[tvoi].astype('str')
mask = (dfraw[tvoi].str.len() == 6) 
dfraw = dfraw.loc[mask]
#removing all the white spaces
dfraw[tvoi]=dfraw[tvoi].str.strip()

dfraw[tvoi]=pd.to_datetime(dfraw[tvoi], format='%M:%S')
dfraw['Date'] = dfraw.apply(lambda row: datetime(row['Year'], row['Month'], row['Day']), axis=1)

#Python activity filter 
#e' qui che viene creato un df che ha un index temporale
pythondf=dfraw[(dfraw['Activity'].str.contains("Python",na=False)) | (dfraw['Activity'].str.contains("python",na=False))] 


## Resampling

In [ ]:
#try better df 
#un dataframe per provare a riaggregare i nostri dati
#per conteggiare i giorni dedicati a python e le ore fatte
#

#We create a copy of 
#the dataframe containing only the 
#python records and
#we aggregate on daily basis 
tb_df=pythondf.copy()
tb_df=pythondf.groupby('Date').count()

rs_df=tb_df
#We resample the dataframe in order to have 
#a continues time series 
#with daily frequency
rs_df=rs_df.resample('D').asfreq().fillna(0)
#divide by 2 because each pomodoro record lasts 25 minutes
#and we create a new column to keep track of that 
rs_df['Cum']=rs_df['Activity'].cumsum().divide(2)

rs_df['Costant_CUM']=3
rs_df['Costant_CUM']=rs_df['Costant_CUM'].cumsum()


#Hours Remaining to be an expert

In [ ]:
#Max Hours 
max_hours=rs_df['Cum'][-1]
print("Total Hours until now",max_hours)
remaining_h=h-max_hours
print("Remaining hours to became an expert: \n",remaining_h)

#Plotting Functions

In [ ]:
#We create a function 
#in order to plot all the hours dedicated to the 
#Variable of Interest (in this specific case python)
#And we compare with the "hours per day  goal"

def practice_plot(df):
    def plotting(df):
        fig, ax= plt.subplots(figsize=(15,10))
        ax.plot(df.index,df['Cum'])
        ax.plot(df.index,df['Costant_CUM'],'g--')
        ax.tick_params(axis='x',rotation=90)
        ax.set(title='Total Hours Studying and Working With Python (%d H)'%(maxh))
        plt.show()
        return 
    
    if len(df)>180:
        maxh=df.Cum.max()
        maxc=df.Costant_CUM.max()
        print("Max Hours %d h"%maxh)

        delta=maxh-maxc
        print("Il differenziale assoluto è di %s ore"%delta)
    
        delta_mens=[-30,-90,-180]
        delta_mens_dic={"Mese":-30,"Trimestre":-90,"Semestre":-180}
        z=df['Cum'].iloc[-1]
        y=df['Costant_CUM'].iloc[-1]

        percentage_change=[(((z-y)/(df['Cum'].iloc[x]-df['Costant_CUM'].iloc[x]))*100).round(2) 
                   for x in delta_mens]
        x_print=[print("\n La percentuale di cambiamento è di %s negli ultimi %s giorni "%(percentage_change[ii],delta_mens[ii])) for ii in [0,1,2]]
        plotting(df)
    elif len(df)>90:
        maxh=df.Cum.max()
        maxc=df.Costant_CUM.max()
        delta_mens=[-30,-90]
        delta_mens_dic={"Mese":-30,"Trimestre":-90}
        z=df['Cum'].iloc[-1]
        y=df['Costant_CUM'].iloc[-1]
        
        percentage_change=[(((z-y)/(df['Cum'].iloc[x]-df['Costant_CUM'].iloc[x]))*100).round(2) 
                   for x in delta_mens]
        x_print=[print("\n La percentuale di cambiamento è di %s negli ultimi %s giorni "%(percentage_change[ii],delta_mens[ii])) for ii in [0,1]]
        plotting(df)
        plotting(df)
    elif len(df)<90:
        maxh=df.Cum.max()
        maxc=df.Costant_CUM.max()
        plotting(df)
    
def m_practice_plot(df): 
    
    maxh=df.Cum.max()
    maxc=df.Costant_CUM.max()
    print("Max Hours %d h"%maxh)
    delta=maxh-maxc

    fig, ax= plt.subplots(figsize=(15,10))
    ax.plot(df.index,df['Cum'])
    ax.plot(df.index,df['Costant_CUM'],'g--')
    ax.tick_params(axis='x',rotation=90)
    ax.set(title='Total Hours Studying and Working With Python (%d H)'%(maxh))
    
    plt.show()
    return     

#Since Ever Plot

In [ ]:
practice_plot(rs_df)

#Current Year Analysis

In [ ]:
#we create a copy in order to modify without 
#corrupting the original df
#containing all the records 
#of the selected activity
#grouped by day 
pythondf=rs_df
#this is a test for timeseries slicing
print("Test_selezione \n", pythondf['2020'].tail())
print(pythondf.head())
print(pythondf.index[1:10])
#here you can put automatically the year
select_Year='2020'
start=-365
print(len(pythondf.iloc[start:]))

In [ ]:

tot_hours=rs_df[select_Year]['Costant_CUM'][-1]-rs_df[select_Year]['Costant_CUM'][0]
average=(tot_hours/365/2).round(2)

print((tot_hours/2).round(2))
print(average,'\n')



#-365 Plot 

In [ ]:
practice_plot(rs_df.iloc[start:])

#Current Year Analysis 

In [ ]:
#we create a 
#current day vector 
#current month vector (int)
#current year vector (int)
today=datetime.now()
actual_month=today.month
actual_year=today.year
print(type(actual_year))

#we convert to string 
#current year vector 
year_str=str(actual_year)
d=1
#we create
#a starting point 
#for current year  
date2=datetime(actual_year,1,1)

#we create a 
#current month threshold 
filtering_threshold=datetime(actual_year,actual_month,d)
#we filter 
#the python activities df
#based on the current month 
python_act_m=pythondf[pythondf.index>=filtering_threshold]
#we filter 
#the python activities df
#based on the current year
python_act_y=pythondf[pythondf.index>=year_str]
#we calculate
#the numbers of days 
#from
#the beginning of the month 
#and the current day 
d_day=(today-filtering_threshold).days
#we calculate
#the numbers of days 
#from
#the beginning of the year
#and the current day 
d_y_day=(today-date2).days

print('Python monthly head \n', python_act_m.head())
print('Python yearly head \n', python_act_y.head())


In [ ]:
practice_plot(python_act_y)

In [ ]:
m_practice_plot(python_act_m)

In [ ]:
#This is a relative plot in order 
#to analyze the learning trend 
#relative to 
#the current year 
#current month 
def relative_plot(df,date): 
    f_df=df[date::].copy()
    f_df['R_Cum']=f_df['Activity'].cumsum().divide(2)
    

    
    f_df['R_Costant_CUM']=np.arange(3,len(f_df)*3+1,3)

    maxh=f_df.R_Cum.max()
    maxc=f_df.R_Costant_CUM.max()
    
    print("Lo studio di Python Da",date," è di %s h"%maxh)

    delta=maxh-maxc
    print("\n Il differenziale relativo è di %s ore"%delta)
    
    fig, ax= plt.subplots()
    ax.plot(f_df.index,f_df['R_Cum'])
    ax.plot(f_df.index,f_df['R_Costant_CUM'],'g--')
    ax.tick_params(axis='x',rotation=90)
    ax.set(title='Relative Hours Studying and Working With Python (%d H)'%(maxh))
    
    plt.show()
    return 

In [ ]:
year='2020'
relative_plot(rs_df,year)

In this section we analyze how we are performing on monthly basis and on yearly basis
We should select manually the month and the year, this way is also possible to go back to any month 

In [ ]:
month='2020-03'

In [ ]:
relative_plot(rs_df,month)
m_practice_plot(python_act_m)

In [ ]:
practice_plot(python_act_y)
relative_plot(rs_df,year)

Da questa sezione iniziamo a valutare ed identiicare la stazionarietà delle nostre serie storiche per poter predire l'opportuno risultato di ore che verranno dedicate nel futuro 

In [ ]:
activity_divided=rs_df.Activity.divide(2)
print(activity_divided.mean())
print(activity_divided.std())
print(activity_divided.median())
print(activity_divided.describe())

In [ ]:
fig, ax=plt.subplots()
ax.hist(activity_divided,bins=24)
ax.set(xlabel='Hours per day',title='Hours per Day Histogram')
plt.show()

In [ ]:
#we try to analyze the difference between days in order to see ho day change 
diff=activity_divided.diff().dropna()

fig, ax=plt.subplots(2,1)
ax[0].plot(diff.index,diff)
ax[0].set(xlabel='Hours per day',title='Hours per Day Histogram')
plt.setp(ax[0].get_xticklabels(),rotation=90)
ax[1].plot(activity_divided.index,activity_divided)
ax[1].set(xlabel='Hours per day',title='Hours per Day Histogram')
plt.setp(ax[1].get_xticklabels(),rotation=90)
plt.show()

In [ ]:
#here we plot the histogram of the difference in activity day 
print("Diff Summary Statistics: \n",diff.describe())
fig, ax=plt.subplots()
ax.hist(diff,bins=48)
ax.set(xlabel='Difference',title='Difference Hours Committed per Day Histogram')
plt.show()

In [ ]:
#we create 
#a normal distribution 
#with 
#mean equal to the average hours per day 
#std is calculated also on daily basis
n_mean=rs_df.Activity.divide(2).mean()
n_std=rs_df.Activity.divide(2).mean()
n_obj=scipy.stats.norm(n_mean, n_std).cdf(n_mean)
print(n_obj)
pdf=[0.025, 0.25, 0.5,0.75, 0.975]
pdf2=[2, 25, 50,75, 97]
#for i in pdf2:
 #   print(i,scipy.stats.norm(n_mean, n_std).pdf(i)

          
normal_obj=np.random.normal(n_mean, n_std,1000)
normal_sorted=sorted(normal_obj)
percentile=[2.5,25,50, 75,97.5]
prc_array=np.percentile(normal_sorted,percentile)
print("percentile array \n", prc_array)
print(prc_array.size)
remaining_days=np.full(prc_array.size,remaining_h)
print("Year remaining to be an expert \n",(remaining_days/prc_array)/365)

Alcune risorse in merito alla:
1) Distribuzione Normale

2) Distribuzione Beta 

3) Distribuzione PERT 

https://www.vosesoftware.com/riskwiki/Beta4distribution.php

https://it.wikipedia.org/wiki/Distribuzione_Beta

https://en.wikipedia.org/wiki/PERT_distribution

https://stats.stackexchange.com/questions/68983/beta-distribution-fitting-in-scipy

https://medium.com/w2hds/beta-distribution-what-when-how-c5ebaafdeedd

https://bizfluent.com/how-6722338-calculate-pert.html

https://math.stackexchange.com/questions/521575/difference-between-probability-and-probability-density

In [ ]:
#in this section we identify autocorrelation for our time series
#
# Import the plot_acf module from statsmodels
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf


In [ ]:
#Try to approximate the actual curve with polynomial features 
#Is a trivial way to make a prediction 
#Just for practice
#not for pratical use 
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from statsmodels.tsa.stattools import adfuller

In [ ]:
print(rs_df.head(5))
print(today)

In [ ]:
degree=1
model = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
x=np.arange(0,len(rs_df.index))
x=x[:,np.newaxis]
y=np.array(rs_df['Cum'].values)
y=y[:,np.newaxis]
print(len(x))
print(len(rs_df['Cum']))
print(len(x),x.shape)
print(len(y),y.shape)
fitted=model.fit(x,y)
x_pred=np.arange(0,len(rs_df.index)+365)
x_pred=x_pred[:,np.newaxis]
y_plot = model.predict(x_pred)
print(type(model))
print(type(fitted))
#we need to get another y pred 
#in order to evaluate 
#the residuals 
y_res=model.predict(x)
residuals=y_res-y
flat_residuals=residuals.flatten()
print(adfuller(flat_residuals))

In [ ]:
#thorugh statsmodel ordinary least squared 
#we extract automatically residuals 
#and test if they are stationary or not 
import statsmodels.api as sm
#
X = sm.add_constant(x)
model_stat = sm.OLS(y,X)
results_stats = model_stat.fit()
print(results_stats.params)
#we test if residuals are stationary or not 
#with adfuller test
print(adfuller(results_stats.resid))

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.plot(x, y)
ax.plot(x_pred,y_plot)
print(y_plot[-1])
average=y_plot[-1]/len(y_plot)
print(np.round(average,2))

In [ ]:
week_moving_average=rs_df.Activity.divide(2).rolling(window=30).mean()
mean_activity=rs_df.Activity.divide(2).mean()
print(mean_activity)
x_moving=np.arange(0,len(week_moving_average))
fig,ax=plt.subplots()
ax.plot(x_moving,week_moving_average)
plt.show()

In this section we try to undestand if the difference between daily records is stationary with ad adfuller test 

In [ ]:
from statsmodels.tsa.stattools import adfuller
result=adfuller(diff)

print(result[0])
print("Adfuller p value:\n",np.round(result[1],4))

###### 